In [1]:
import statsmodels.formula.api as smf
import pandas as pd

from eeg_analyzer import EEGAnalyzer
from utils.config import DATASETS

ANALYZER_NAME = "eeg_analyzer_test"

# Trying to load the EEGAnalyzer
analyzer = EEGAnalyzer.load_analyzer(ANALYZER_NAME)
if analyzer is None:
    print(f"Analyzer {ANALYZER_NAME} not found. Creating a new one.")
    analyzer = EEGAnalyzer(DATASETS, ANALYZER_NAME)
    analyzer.save_analyzer()

[EEGAnalyzer - eeg_analyzer_test] EEGAnalyzer state loaded from C:\Users\si_ve\Documents\Master_AttentionalDirectionResearch\data\eeg_analyzer_derivatives\eeg_analyzer_test\analyzer_state.pkl


In [3]:
# Creating a DataFrame with the data
df = analyzer.create_dataframe()

# Printing the first few rows of the DataFrame
print("DataFrame head:")
df.head()

[EEGAnalyzer - eeg_analyzer_test] Creating DataFrame from datasets...
[EEGAnalyzer - eeg_analyzer_test] Processing dataset: braboszcz2017
[EEGAnalyzer - eeg_analyzer_test] Processing dataset: jin2019
[EEGAnalyzer - eeg_analyzer_test] Processing dataset: touryan2022
[EEGAnalyzer - eeg_analyzer_test] DataFrame created with 5165632 rows and 14 columns.
DataFrame head:


,dataset,subject_session,subject_id,session_id,group,epoch_idx,task,task_orientation,state,channel,cortical_region,hemisphere,band_power,log_band_power
0,braboszcz2017,025_1,025,1,ctr,5,med2,internal,OT,Fp1,prefrontal,left,6.109082,1.809776
1,braboszcz2017,025_1,025,1,ctr,5,med2,internal,OT,AF7,prefrontal,left,5.516391,1.707724
2,braboszcz2017,025_1,025,1,ctr,5,med2,internal,OT,AF3,prefrontal,left,6.665417,1.896932
3,braboszcz2017,025_1,025,1,ctr,5,med2,internal,OT,F1,fronto-parietal,left,8.139580,2.096739
4,braboszcz2017,025_1,025,1,ctr,5,med2,internal,OT,F3,fronto-parietal,left,7.792555,2.053169


In [4]:
# Fitting a linear mixed effects model
model = smf.mixedlm(
    "log_band_power ~ C(state) * C(task_orientation)",
    df,
    groups=df["dataset"],
    re_formula="1",  # Random intercept per dataset
    vc_formula={
        "subject": "0 + C(subject_id)",
        "session": "0 + C(session_id)",
        "channel": "0 + C(channel)",
        "channel_state": "0 + C(channel):C(state)"
    }
)

result = model.fit()
print(result.summary())

MemoryError: Unable to allocate 6.38 GiB for an array with shape (3806656, 225) and data type float64